# Malu's way: criando o dataset que servirá de base para o projeto.
# Clarificações: o material de estudo usado como base foi Braunwald-Heart-Disease-A-Textbook-of-Cardiovascular-Medicine-6th-ed. pdf
# A partir deste livro, foram selecionadas frases sobre diagnósticos que montaram o texto que será processado (medBook.txt - perdão pelo nome ridículo de arquivo).

In [ ]:
# Começamos importando a biblioteca queridinha do python para análise e manipulação de dados
import pandas as pd

# Lista de sintomas cardiovasculares mais comuns, de acordo com Braunwald-Heart-Disease-A-Textbook-of-Cardiovascular-Medicine-6th-ed
cardio_symptoms = [
    "chest_pain","chest_tightness","angina","shortness_of_breath","dyspnea",
    "orthopnea","paroxysmal_nocturnal_dyspnea","palpitations","tachycardia","arrhythmia",
    "fatigue","weakness","dizziness","syncope","lightheadedness","sweating",    
    "edema","ankle_swelling","nausea","vomiting"
]

# Cria um DataFrame vazio com colunas que são os sintomas, e que possuem valores booleanos
df = pd.DataFrame(columns=cardio_symptoms).astype(bool)

# Adiciona a coluna extra "diagnostic" (string inicialmente)
df["diagnostic"] = pd.Series(dtype="str")

# Salva em CSV
df.to_csv("CardioSymptomsDataset.csv", index=False, encoding="utf-8")
 
# Eu sempre vou printando tudo como forma de debugar o código
print("Dataset vazio criado com sucesso!")
print(df.dtypes)  # mostra os tipos de dados
print(df.head())  # mostra primeiras linhas (vazio ainda)

Dataset vazio criado com sucesso!
chest_pain                        bool
chest_tightness                   bool
angina                            bool
shortness_of_breath               bool
dyspnea                           bool
orthopnea                         bool
paroxysmal_nocturnal_dyspnea      bool
palpitations                      bool
tachycardia                       bool
arrhythmia                        bool
fatigue                           bool
weakness                          bool
dizziness                         bool
syncope                           bool
lightheadedness                   bool
sweating                          bool
edema                             bool
ankle_swelling                    bool
nausea                            bool
vomiting                          bool
diagnostic                      object
dtype: object
Empty DataFrame
Columns: [chest_pain, chest_tightness, angina, shortness_of_breath, dyspnea, orthopnea, paroxysmal_nocturnal_dyspnea,

# Agora, processaremos o arquivo com diagnósticos anteriores e guardaremos em um dicionários com as doenças que forem sendo achadas como chave e um hashset de sintomas como valor (assim, evitamos repetições e otimizamos o código, afinal, as consultas em ambas essas estruturas de dados é O(1)).
# Esse dicionário servirá de base para popular o dataset que fizemos antes!
# Usaremos nosso arquivo de nlp para extrair as entidades (doença, sintoma)

In [5]:
# Biblioteca de processamento de linguagem natural
import spacy
# Função que divide textos em sentenças e os tokeniza. Sem a tokenização, não é possível fazer o processamento da LN
from nltk.tokenize import sent_tokenize
from nlp import NLPProcessor

text = "medBook.txt"

sentences = sent_tokenize(text)

# Carrega modelo spaCy pequeno, em inglês
nlp = spacy.load("en_core_web_sm")
processor = NLPProcessor(nlp)

# Dicionário para armazenar doença -> conjunto de sintomas
disease_to_symptoms = dict()

with open(text, "r", encoding="utf-8") as f:
    print("Entidades extraídas:")
    for line in f:
        # Remove aspas extras e espaços, que é o padrão de organização do nosso arqiuvo
        sentence = line.strip().strip('"')
        
        # Checa se estão negadas – por exemplo, se a frase diz "sem dor no peito" ou "não apresenta febre",
        # a função marca essas entidades como negadas, evitando incluí-las como sintomas ou doenças presentes.
        entities = processor.extract_entities_with_negation(sentence)
        print(entities)
        
        # Filtra sintomas e doenças não negados
        symptoms = {e[0] for e in entities if e[1] == "SYMPTOM" and not e[2]}
        diseases = [e[0] for e in entities if e[1] == "DISEASE" and not e[2]]
        
        # Itera sobre cada doença encontrada e vai adicionando os sintomas nas doenças
        for disease in diseases:
            if disease not in disease_to_symptoms:
                disease_to_symptoms[disease] = set()
            disease_to_symptoms[disease].update(symptoms)

# Olha eu debugando aqui de novo
print("Dicionário de doenças e sintomas extraídos:")
for disease, symptoms in disease_to_symptoms.items():
    print(f"{disease}: {symptoms}")

Entidades extraídas:
[]
[('coronary artery disease', 'DISEASE', False), ('chest pain', 'SYMPTOM', False), ('fatigue', 'SYMPTOM', False)]
[('Shortness of breath', 'SYMPTOM', False), ('coronary artery disease', 'DISEASE', False)]
[('coronary artery disease', 'DISEASE', False), ('palpitations', 'SYMPTOM', False)]
[('coronary artery disease', 'DISEASE', False), ('sweating', 'SYMPTOM', False)]
[('coronary artery disease', 'DISEASE', False), ('Fatigue', 'SYMPTOM', False)]
[('shortness of breath', 'SYMPTOM', False), ('chest pain', 'SYMPTOM', False), ('myocardial infarction', 'DISEASE', False)]
[('myocardial infarction', 'DISEASE', False), ('Sweating', 'SYMPTOM', False)]
[('myocardial infarction', 'DISEASE', False), ('Palpitations', 'SYMPTOM', False)]
[('myocardial infarction', 'DISEASE', False), ('fatigue', 'SYMPTOM', False)]
[('myocardial infarction', 'DISEASE', False), ('Dyspnea', 'SYMPTOM', False)]
[('chest pain', 'SYMPTOM', False), ('angina pectoris', 'DISEASE', False)]
[('Shortness of br